In [9]:
from tdc.generation import MolGen
from tdc.single_pred import ADME
from tdc.single_pred import Tox
from tdc.utils import retrieve_label_name_list
import pandas as pd

path = "/home/bhux/workplace/drug_discovery/data"

def get_splits(data): 
    split = data.get_split()
    split = [split['train'], split['valid'], split['test']]
    split = pd.concat(split)
    return split

adme_names = [
    'Caco2_Wang',   # 906
    'Lipophilicity_AstraZeneca',
    'Solubility_AqSolDB',
    'HydrationFreeEnergy_FreeSolv',
    'PPBR_AZ',
    'VDss_Lombardo',
    'Half_Life_Obach',
    'Clearance_Hepatocyte_AZ',
    'Clearance_Microsome_AZ',
]

tox_names = [
    'LD50_Zhu',
]

data_dfs = {}
for name in adme_names:
    data = ADME(name = name, path=path)
    data_dfs[name] = get_splits(data)

data = Tox(name = 'LD50_Zhu', path=path)
data_dfs['LD50_Zhu'] = get_splits(data)

label_list = retrieve_label_name_list('herg_central')
for lname in label_list[:-1]: # no inhib
    data = Tox(name = 'herg_central', label_name = lname)
    data_dfs['herg_central_'+lname] = get_splits(data)

print(data_dfs.keys())

# categorical adme 
cat_adme_names = [
    # 'PAMPA_NCATS',
    # 'HIA_Hou',
    # 'Pgp_Broccatelli',
    # 'Bioavailability_Ma',
    # 'BBB_Martins',
    # 'CYP2C19_Veith',
    # 'CYP2D6_Veith',
    # 'CYP3A4_Veith',
    # 'CYP1A2_Veith',
    # 'CYP2C9_Veith',
    # 'CYP2C9_Substrate_CarbonMangels',
    # 'CYP2D6_Substrate_CarbonMangels',
    # 'CYP3A4_Substrate_CarbonMangels',
]

for name in cat_adme_names:
    data = ADME(name = name, path=path)
    data_dfs[name] = get_splits(data)

# cat_tox_names = [
#     'AMES',
#     'DILI',
#     'Skin Reaction',
#     'Carcinogens_Lagunin',
#     'ClinTox',
# ]

# for name in cat_tox_names:
#     data = Tox(name = name, path=path)
#     data_dfs[name] = get_splits(data)

# # Tox21 and ToxCast
# name = 'Tox21'
# label_list = retrieve_label_name_list(name)
# for lname in label_list: # no inhib
#     data = Tox(name = name, label_name = lname)
#     data_dfs[name+lname] = get_splits(data)

# name = 'Toxcast'
# label_list = retrieve_label_name_list(name)
# for lname in label_list[:10]: # no inhib
#     data = Tox(name = name, label_name = lname)
#     data_dfs[name+lname] = get_splits(data)

Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!
Found local copy...
Loading...
Done!


dict_keys(['Caco2_Wang', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB', 'HydrationFreeEnergy_FreeSolv', 'PPBR_AZ', 'VDss_Lombardo', 'Half_Life_Obach', 'Clearance_Hepatocyte_AZ', 'Clearance_Microsome_AZ', 'LD50_Zhu', 'herg_central_hERG_at_1uM', 'herg_central_hERG_at_10uM'])


In [10]:
import numpy as np

# Populate smiles
smiles = []
for _, df in data_dfs.items():
    smiles.extend(list(df['Drug']))
smiles = list(set(smiles))
print(len(smiles))

# Build dataset
lengths = []
dataset = {'Drug': smiles}
for k, df in data_dfs.items():
    df = {v['Drug']: v['Y'] for v in df.to_dict(orient='records')}
    dataset[k] = []
    for smile in smiles:
        dataset[k].append(df.get(smile, np.nan))
    lengths.append(len(dataset[k])-sum(np.isnan(np.array(dataset[k]))))

print(dataset.keys())

df = pd.DataFrame(data=dataset)

display(df[:10])



327533
dict_keys(['Drug', 'Caco2_Wang', 'Lipophilicity_AstraZeneca', 'Solubility_AqSolDB', 'HydrationFreeEnergy_FreeSolv', 'PPBR_AZ', 'VDss_Lombardo', 'Half_Life_Obach', 'Clearance_Hepatocyte_AZ', 'Clearance_Microsome_AZ', 'LD50_Zhu', 'herg_central_hERG_at_1uM', 'herg_central_hERG_at_10uM'])


,Drug,Caco2_Wang,Lipophilicity_AstraZeneca,Solubility_AqSolDB,HydrationFreeEnergy_FreeSolv,PPBR_AZ,VDss_Lombardo,Half_Life_Obach,Clearance_Hepatocyte_AZ,Clearance_Microsome_AZ,LD50_Zhu,herg_central_hERG_at_1uM,herg_central_hERG_at_10uM
0,Cc1n[nH]c(C)c1-c1nc2ccccc2[nH]1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.2275,0.97548
1,C(#Cc1ccccc1)C[N+]1(CC#Cc2ccccc2)CCCC1.[Br-],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-17.4346,-55.52842
2,Br.N=C1c2ccccc2CN1NC(=O)c1cccc(S(=O)(=O)N2CCCC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.4922,13.45668
3,C=CCSc1nnc(NC(=O)c2ccc(S(=O)(=O)NCC3CCCO3)cc2)s1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0481,13.52548
4,O=C(CS(=O)(=O)c1cn(Cc2ccc(F)cc2)c2ccccc12)N1CC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8734,-16.65112
5,CCc1cccc(NC(=O)c2cccc(S(=O)(=O)N3CCN(C)CC3)c2)c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.1511,-53.90882
6,CCOCCCNC(=O)c1cn(CC)c2ccc(S(=O)(=O)N3CCC(C)CC3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.2509,-22.63932
7,CN(CC(=O)NCC1CCCO1)S(=O)(=O)c1cc(Cl)ccc1Cl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9166,-0.78322
8,CCCCn1c(O)c(/C=C2/C=CC=N2)c(=O)[nH]c1=O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.2141,6.34478
9,CC(C)C(=O)OCC(=O)Nc1ccc(N=Nc2ccccc2)cc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.8161,4.20108


In [11]:
from collections import Counter

combinations = []
for row in df.drop(labels=['Drug', 'herg_central_hERG_at_10uM'], axis=1).iterrows():
    combinations.append("".join([str(int(np.isnan(v) == False)) for _,v in list(row)[1].items()]))

uc = Counter(combinations)
print(uc.keys())
print(uc.values())

dict_keys(['00000000001', '00100000000', '00100000011', '01000000101', '00000000010', '01000000000', '00100000010', '00000000011', '00000100010', '00001000000', '00100000001', '10000000000', '00000010000', '00000010001', '01001000000', '01000001100', '10000010000', '00000100000', '01001001000', '01101110011', '01101001111', '00001000100', '00001001100', '10000000010', '01001001100', '01000000100', '01000000001', '00110000010', '00110000011', '00000100001', '10100000000', '00110000000', '00000001000', '00010000000', '01001000001', '01000001000', '00100000100', '01110000010', '10110000010', '00000100011', '01100000000', '01000010000', '01100000010', '01001000100', '00000001100', '00001000001', '01101011001', '00000001101', '01000001101', '00101000010', '00000110000', '11101000011', '01001001101', '00001001000', '00100010000', '01001111000', '01100000011', '01101011110', '10000011110', '01100010011', '00101000000', '01001010010', '00001100000', '11100000010', '01000100000', '11001010000',

In [12]:
cbool = np.array(combinations) != '0'*10 + '1' + '0'*(len(list(uc.keys())[0])-11) #'00000000001000000000000000000000000000000'
inc = np.random.choice([i for i, x in enumerate(cbool) if x == False], 7900, replace=False)
cbool = [True if i in inc else v for i,v in enumerate(cbool)]

fcomb = np.array(combinations)[cbool]
fdf = df[cbool]

print(len(fdf.index))

30000


In [13]:
def unison_shuffled_copies(a, b):
    #assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], p

rcomb, perm = unison_shuffled_copies(fcomb, fdf.iloc)
lengths[10] = 7900
del lengths[11]
valCount = np.array(lengths)*0.1

trIdx = []
valIdx = []
for c, p in zip(rcomb, perm):
    inc = False
    for i, j in enumerate(list(c)):
        if j == "1" and valCount[i] > 0:
            valCount[i] -= 1
            inc = True
    if inc:
        valIdx.append(p)
    else:
        trIdx.append(p)

print(len(trIdx))
print(len(valIdx))

27039
2961


In [14]:
trdf = fdf.iloc[trIdx]
valdf = fdf.iloc[valIdx]

In [15]:
from transformers import AutoTokenizer, AutoModel, T5EncoderModel, AutoModelForMaskedLM
import torch
from tqdm import tqdm

model = "sagawa/PubChem-10m-deberta"
# "sagawa/PubChem-10m-deberta"
# "seyonec/ChemBERTa-zinc-base-v1"
# "DeepChem/ChemBERTa-10M-MLM"
# "sagawa/PubChem-10m-t5-v2"

# "ncfrey/ChemGPT-4.7M"
# "RaphaelMourad/Mistral-Chem-v1-417M"
# "jonghyunlee/ChemBERT_ChEMBL_pretrained"

# Initialize tokenizer and model for drug embeddings
drug_tokenizer = AutoTokenizer.from_pretrained(model)
drug_model = AutoModel.from_pretrained(model)

drug_model.to('cuda')

# Function to get drug embeddings from SMILES strings
def get_drug_embeddings(smiles_list, tokenizer, model, batch_size=64):
    embeddings = []
    for i in tqdm(range(0, len(smiles_list), batch_size)):
        batch = smiles_list[i:i+batch_size]
        encoding = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        input_ids = encoding["input_ids"].to('cuda')
        attention_mask = encoding["attention_mask"].to('cuda')  # Get the attention mask
        
        with torch.no_grad():
            # Pass both input_ids and attention_mask to the model
            output = model(input_ids, attention_mask=attention_mask)
            # You can use either the last hidden state or the pooled output
            # depending on your requirements. Here we're using the mean of the last hidden state.
            embedding = output.last_hidden_state.mean(1)
            embeddings.extend(embedding.cpu().numpy())
            
    return embeddings

In [16]:
# Save train
drug_embeddings_split_kd = get_drug_embeddings(list(fdf['Drug']), drug_tokenizer, drug_model)
print(len(drug_embeddings_split_kd))
np.save("./data/xtended_emb_all_deberta_pubchem", np.array(drug_embeddings_split_kd))
fdf.to_csv("./data/xtended_data_all.csv", index=False)

100%|██████████| 469/469 [00:47<00:00,  9.91it/s]


30000


In [17]:
drug_embeddings_split_kd = get_drug_embeddings(list(valdf['Drug']), drug_tokenizer, drug_model)
print(len(drug_embeddings_split_kd))
np.save("./data/xtended_emb_val_deberta_pubchem", np.array(drug_embeddings_split_kd))
valdf.to_csv("./data/xtended_data_val.csv", index=False)

100%|██████████| 47/47 [00:04<00:00, 11.70it/s]

2961
